# BIAFlows

## Nuclei Tracking 2D+t

### Fiji-workflow

The workflow treats the time-dimension as z-dimension and does a 3D-segmentation of the objects. The resulting object slices are then reduced to a center point.

#### Initialize the cytomine client and job

The cytomine client needs some parameters to connect to the server. The host on which the server is running, the public and private keys to make a connection, the id of the project and the id of the software.
The remaining 3 parameters will be passed on to the image analysis workflow.

In [1]:
from getpass import getpass 

publicKey = getpass("Please enter the public key: ")
privateKey = getpass("Please enter the private key: ")

argv = ['--cytomine_public_key', publicKey, 
 '--cytomine_host',  'neubias.cytomine.be',
 '--cytomine_private_key',  privateKey,
 '--cytomine_id_project',  '1695226', 
 '--cytomine_id_software', '3804897',
 '--ij_gauss_radius',  '3',
 '--ij_threshold',  '60',
 '--ij_open_radius', '7']

Please enter the public key: ········
Please enter the private key: ········


Import CytomineJob and Job and update the status information.

In [2]:
from cytomine import CytomineJob
from cytomine.models import Job
from beakerx.object import beakerx
jobID=-666
with CytomineJob.from_cli(argv) as cj:
        cj.job.update(status=Job.RUNNING, progress=0, statusComment="Initialisation...")
        jobID = cj.job.id
        beakerx.ij_gauss_radius = cj.parameters.ij_gauss_radius
        beakerx.ij_threshold = cj.parameters.ij_threshold
        beakerx.ij_open_radius = cj.parameters.ij_open_radius

[2019-01-23 17:48:33,603][INFO] [GET] [currentuser] CURRENT USER - 5739 : neubias | 200 OK
[2019-01-23 17:48:33,665][INFO] [GET] [project] 1695226 : NEUBIAS-BENCHMARK-NUCLEI-TRACKING-NODIVISION | 200 OK
[2019-01-23 17:48:33,723][INFO] [GET] [software] 3804897 : NucleiTracking-ImageJ | 200 OK
[2019-01-23 17:48:33,774][INFO] [GET] [softwareparameter collection] 8 objects | 200 OK
[2019-01-23 17:48:33,907][INFO] [POST] [job] 5873138 : None | 200 OK
[2019-01-23 17:48:33,965][INFO] [GET] [user] 5873144 : JOB[neubias ], 2019-01-23 16:48:33:857 | 200 OK
[2019-01-23 17:48:34,017][INFO] [GET] [currentuser] CURRENT USER - 5873144 : JOB[neubias ], 2019-01-23 16:48:33:857 | 200 OK
[2019-01-23 17:48:34,018][INFO] Job (id:5873138) status update: "None" (status: RUNNING, progress: 0%)
[2019-01-23 17:48:34,106][INFO] [PUT] [job] 5873138 : None | 200 OK
[2019-01-23 17:48:34,172][INFO] [POST] [jobparameter] 5873152 : ij_gauss_radius | 200 OK
[2019-01-23 17:48:34,239][INFO] [POST] [jobparameter] 5873158 

#### Create local directories

Create the local working directories in a subfolder jobID of the user's home folder.

- in: input images
- out: output images
- ground_truth: ground truth images
- tmp: temporary path

In [3]:
import os
from beakerx.object import beakerx
base_path = "{}".format(os.getenv("HOME"))
gt_suffix = "_lbl"
working_path = os.path.join(base_path, str(jobID))
in_path = os.path.join(working_path, "in")
out_path = os.path.join(working_path, "out")
gt_path = os.path.join(working_path, "ground_truth")
tmp_path = os.path.join(working_path, "tmp")

if not os.path.exists(working_path):
    os.makedirs(working_path)
    os.makedirs(in_path)
    os.makedirs(out_path)
    os.makedirs(gt_path)
    os.makedirs(tmp_path)

beakerx.in_path = in_path
beakerx.out_path = out_path

#### Download the images

In [4]:
from cytomine.models import ImageGroupCollection
cj.job.update(progress=10, statusComment="Downloading images (to {})...".format(in_path))
image_group = ImageGroupCollection().fetch_with_filter("project", cj.parameters.cytomine_id_project)

input_images = [i for i in image_group if gt_suffix not in i.name]
gt_images = [i for i in image_group if gt_suffix in i.name]

for input_image in input_images:
    input_image.download(os.path.join(in_path, "{id}.tif"))

for gt_image in gt_images:
    related_name = gt_image.name.replace(gt_suffix, '')
    related_image = [i for i in input_images if related_name == i.name]
    if len(related_image) == 1:
        gt_image.download(os.path.join(gt_path, "{}.tif".format(related_image[0].id)))

[2019-01-23 17:48:41,584][INFO] Job (id:5873138) status update: "Downloading images (to /home/baecker/5873138/in)..." (status: SUCCESS/TERMINATED, progress: 0%)
[2019-01-23 17:48:41,723][INFO] [PUT] [job] 5873138 : None | 200 OK
[2019-01-23 17:48:41,782][INFO] [GET] [imagegroup collection] 2 objects | 200 OK
[2019-01-23 17:48:42,242][INFO] File downloaded successfully from http://neubias.cytomine.be/api/imagegroup/4042304/download
[2019-01-23 17:48:42,617][INFO] File downloaded successfully from http://neubias.cytomine.be/api/imagegroup/4042708/download


#### Call the image analysis workflow 

In [5]:
from subprocess import call

cj.job.update(progress=25, statusComment="Launching workflow...")
command = "/usr/bin/xvfb-run ./ImageJ-linux64 -macro NucleiTracking.ijm " \
		"\"input={}, output={}, gauss_rad={}, threshold={}, open_rad={}\" -batch".format(in_path, out_path, cj.parameters.ij_gauss_radius, cj.parameters.ij_threshold, cj.parameters.ij_open_radius)

return_code = call(command, shell=True, cwd="./Fiji.app")  # waits for the subprocess to return

if return_code != 0:
    err_desc = "Failed to execute the ImageJ macro (return code: {})".format(return_code)
    cj.job.update(progress=50, statusComment=err_desc)
    raise ValueError(err_desc)
cj.job.update(progress=30, statusComment="Workflow finished...")


[2019-01-23 17:48:52,212][INFO] Job (id:5873138) status update: "Launching workflow..." (status: SUCCESS/TERMINATED, progress: 10%)
[2019-01-23 17:48:52,364][INFO] [PUT] [job] 5873138 : None | 200 OK
[2019-01-23 17:49:00,180][INFO] Job (id:5873138) status update: "Workflow finished..." (status: SUCCESS/TERMINATED, progress: 25%)
[2019-01-23 17:49:00,279][INFO] [PUT] [job] 5873138 : None | 200 OK


#### Create and upload annotations

In [6]:
from neubiaswg5.exporter.mask_to_points import mask_to_points_3d
import io
from skimage import io
from cytomine.models import Annotation,AnnotationCollection, ImageGroupCollection, ImageInstance, ImageSequenceCollection
from itertools import groupby
from shapely.geometry import LineString, Point, MultiPoint
from shapely.affinity import affine_transform
import numpy as np
   
for image in cj.monitor(input_images, start=60, end=80, period=0.1, prefix="Extracting and uploading tracks from point-masks"):
    file = "{}.tif".format(image.id)
    path = os.path.join(out_path, file)
    data = io.imread(path)

    # extract objects
    objects = mask_to_points_3d(data, time=True)

    objects = sorted(objects, key=lambda annotationSlice: annotationSlice.time)    
    objects = sorted(objects, key=lambda annotationSlice: annotationSlice.label)    
    grouped = groupby(objects,  key=lambda annotationSlice: annotationSlice.label)
    
    image_sequences = ImageSequenceCollection().fetch_with_filter("imagegroup", image.id)
    
    depth_to_image = {iseq.time: iseq.image for iseq in image_sequences}
    height = ImageInstance().fetch(image_sequences[0].image).height
    
    collection = AnnotationCollection()
    
    groups = []
    uniquekeys = []
    for k, g in grouped:
        groups.append(list(g))      # Store group iterator as a list
        uniquekeys.append(k)
    
    for key, group in zip(uniquekeys, groups):
        listOfPoints = []
        for oSlice in group:
            listOfPoints.append((oSlice.polygon.x, oSlice.polygon.y))
        # shape = LineString(listOfPoints)    
        shape = MultiPoint(listOfPoints)
        for oSlice2 in group:
            polygon = affine_transform(shape, [1, 0, 0, -1, 0, height])
            annotation = Annotation(location=polygon.wkt, id_image=depth_to_image[oSlice2.time], id_project=cj.parameters.cytomine_id_project, property=[{"key": "index", "value": str(oSlice2.label)}])
            collection.append(annotation)
        
    collection.save()
    print("Found {} objects in this image {}.".format(len(objects), image.id))
    
    

[2019-01-23 17:49:03,590][INFO] Job (id:5873138) status update: "Extracting and uploading tracks from point-masks (1/1)." (status: RUNNING, progress: 30%)
[2019-01-23 17:49:03,754][INFO] [PUT] [job] 5873138 : None | 200 OK
[2019-01-23 17:49:03,941][INFO] [GET] [imagesequence collection] 10 objects | 200 OK
[2019-01-23 17:49:03,998][INFO] [GET] [imageinstance] 4041830 : /1539945624298/ShortMovie.ome_Z0_C0_T5.tiff | 200 OK
[2019-01-23 17:49:10,316][INFO] [POST] algoannotations 5873237,5873251,5873265,5873279,5873295,5873311,5873325,5873339,5873353,5873367,5873381,5873394,5873407,5873420,5873433,5873446,5873459,5873472,5873485,5873498,5873511,5873524,5873537,5873552,5873565,5873578,5873591,5873604,5873617,5873630,5873643,5873656,5873669,5873682,5873695,5873708,5873721,5873734,5873747,5873760,5873773,5873786,5873799,5873812,5873825,5873838,5873851,5873864,5873877,5873890,5873903,5873916,5873929,5873942,5873955,5873968,5873981,5873994,5874007,5874020,5874033,5874046,5874059,5874072,5874085,

[2019-01-23 17:50:07,195][INFO] [POST] algoannotations 5885017,5885032,5885045,5885058,5885071,5885084,5885097,5885110,5885123,5885136,5885149,5885162,5885175,5885188,5885201,5885214,5885227,5885240,5885253,5885266,5885279,5885292,5885305,5885318,5885331,5885344,5885357,5885370,5885383,5885396,5885409,5885422,5885435,5885448,5885461,5885474,5885487,5885500,5885513,5885526,5885539,5885552,5885565,5885578,5885591,5885604,5885617,5885630,5885643,5885656,5885669,5885682,5885695,5885708,5885721,5885734,5885747,5885760,5885773,5885786,5885799,5885812,5885825,5885838,5885851,5885864,5885877,5885890,5885907,5885920,5885933,5885946,5885959,5885972,5885985,5885998,5886011,5886024,5886037,5886050,5886063,5886076,5886089,5886104,5886117,5886130,5886143,5886156,5886169,5886182,5886195,5886208,5886221,5886234,5886247,5886260,5886273,5886286,5886299,5886312 added | 200 OK
[2019-01-23 17:50:13,575][INFO] [POST] algoannotations 5886325,5886338,5886351,5886364,5886377,5886390,5886403,5886416,5886429,588

#### Calculate and upload metrics

In [7]:
from neubiaswg5.metrics import computemetrics_batch
from cytomine.models import Property

cj.job.update(progress=80, statusComment="Computing and uploading metrics...")
outfiles, reffiles = zip(*[
    (os.path.join(out_path, "{}.tif".format(image.id)),
     os.path.join(gt_path, "{}.tif".format(image.id)))
    for image in input_images
])

results = computemetrics_batch(outfiles, reffiles, "PrtTrk", tmp_path)

for key, value in results.items():
    Property(cj.job, key=key, value=str(value)).save()
Property(cj.job, key="IMAGE_INSTANCES", value=str([im.id for im in input_images])).save()

[2019-01-23 17:51:08,277][INFO] Job (id:5873138) status update: "Computing and uploading metrics..." (status: RUNNING, progress: 60%)
[2019-01-23 17:51:08,396][INFO] [PUT] [job] 5873138 : None | 200 OK


FileNotFoundError: [Errno 2] No such file or directory: '/home/baecker/5873138/tmp/intracks.xml.score.txt'

[2019-01-23 17:52:16,118][INFO] Job (id:5873138) status update: "Finished." (status: SUCCESS/TERMINATED, progress: 80%)
[2019-01-23 17:52:16,207][INFO] [PUT] [job] 5873138 : None | 200 OK


#### Cleanup - remove the files and folders that have been downloaded or created.

In [ ]:
for image in input_images:
    file = str(image.id)
    path = out_path + "/" + file + ".tif"
    os.remove(path);
    path = in_path + "/" + file  + ".tif"
    os.remove(path);
    path = gt_path + "/" + file  + ".tif"
    os.remove(path);
    
tmpData = ['intracks.xml', 'intracks.xml.score.txt', 'reftracks.xml']
for file in tmpData:
    path = tmp_path + "/" + file
    os.remove(path);
if os.path.exists(working_path):
    os.rmdir(in_path)
    os.rmdir(out_path)
    os.rmdir(gt_path)
    os.rmdir(tmp_path)
    os.rmdir(working_path)

#### End the job and the connection to cytomine

In [8]:
cj.job.update(status=Job.TERMINATED, progress=100, statusComment="Finished.")
print("END.")